In [1]:
import numpy as np
from osgeo import gdal
import tables
import os
import glob

In [1]:
from General_Raster_Funcs.TiffManagement import SaveLZWTiff

In [ ]:
#nDir = r'C:\Users\zool1301\Documents\Other_Data\Rainfall\GPMM_Imerg\new'
inDir = r'\\129.67.26.176\map_data\hsg\GPMM\Downloaded'
#rmmInDir = r'C:\Users\zool1301\Documents\Other_Data\Rainfall\TRMM_3B43_All_Monthly'
#outDir = r'C:\Users\zool1301\Documents\Other_Data\Rainfall\GPMM_Imerg\Tif'
outDir = r'\\129.67.26.176\map_data\hsg\GPMM\tif'
#trmmOutDir = r'C:\Users\zool1301\Documents\Other_Data\Rainfall\TRMM_3B43_All_Monthly'

In [ ]:
inFiles = glob.glob(os.path.join(inDir,'*.HDF5'))
#trmmInFiles = glob.glob(os.path.join(trmmInDir,'*.HDF'))

In [ ]:
globalProj = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]'
globalGT = (-180.0, 0.1, 0.0, 90, 0.0, -0.1)
trmmGT = (-180.0, 0.25, 0.0, 50, 0.0, -0.25)

In [ ]:
hrsPerMonth = [i*24 for i in [31,28,31,30,31,30,31,31,30,31,30,31]]

In [ ]:
hrsPerMonth

In [ ]:
outDrv = gdal.GetDriverByName('GTiff')
outFNBaseTemplate = "GPMM.IMerg.Monthly.{0}.{1}.{2}.tif"
outFNTemplate = os.path.join(outDir,outFNBaseTemplate)
# the no data value is prob supposed to be =9999.9 but FP error actually makes it
dodgyNDV = -9999.90039062
       
for fn in inFiles:
    
    gpmmFileDate = os.path.basename(fn).split('.')[4][:8]
    gpmmFileVer = os.path.basename(fn).split('.')[6]                                
    gpmmFileMonthNum = int(gpmmFileDate[4:6]) - 1
    
    if int(gpmmFileDate) < 20150400:
        continue
        
    outFNAvg = outFNTemplate.format(gpmmFileDate, gpmmFileVer, "MM_Per_Hr")
    outFNTot = outFNTemplate.format(gpmmFileDate, gpmmFileVer, "MM_Total")
    
    if os.path.exists(outFNAvg):
        continue
    print fn
    t = tables.openFile(fn)
    precipFile = t.root.Grid.precipitation
    precipMem = np.copy(precipFile)
    t.close()
    t = None
    
    # the HDFs are in a weird order, columns and rows are transposed and columns are backwards
    outArr = precipMem.T[::-1, ::1]
    SaveLZWTiff(outArr, dodgyNDV, globalGT, globalProj, )
    outRaster = outDrv.Create(outFNAvg,
                          precipMem.shape[0],precipMem.shape[1],1,gdal.GDT_Float32,
                          ["COMPRESS=LZW","TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES"])
    outRaster.SetGeoTransform(globalGT)
    outRaster.SetProjection(globalProj)
    outBand = outRaster.GetRasterBand(1)
    outBand.SetNoDataValue(dodgyNDV)
    outBand.WriteArray(precipMem.T[::-1,::1])
    outBand.FlushCache()
    outBand = None
    outRaster = None
    
    precipMem[precipMem!=dodgyNDV] *= hrsPerMonth[gpmmFileMonthNum]
    outRaster = outDrv.Create(outFNTot,
                          precipMem.shape[0],precipMem.shape[1],1,gdal.GDT_Float32,
                          ["COMPRESS=LZW","TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES"])
    outRaster.SetGeoTransform(globalGT)
    outRaster.SetProjection(globalProj)
    outBand = outRaster.GetRasterBand(1)
    outBand.SetNoDataValue(dodgyNDV)
    # the HDFs are in a weird order, columns and rows are transposed and columns are backwards
    outBand.WriteArray(precipMem.T[::-1,::1])
    outBand.FlushCache()
    outBand = None
    outRaster = None
    

In [ ]:
outFNBaseTemplate = "TRMM_3B43_Monthly_{0}_{1}.tif"
outFNTemplate = os.path.join(trmmOutDir,outFNBaseTemplate)
outDrv = gdal.GetDriverByName('GTiff')
for fn in trmmInFiles:
    dsRef = "".join(["HDF4_SDS:UNKNOWN:",
                     fn,
                     ":0"])
    
    d = gdal.Open(dsRef)
    b = d.GetRasterBand(1)
    precipMem = b.ReadAsArray()
    gpmmFileDate = os.path.basename(fn).split('.')[1]                                
    gpmmFileVer = os.path.basename(fn).split('.')[2]                                
    
    outFN = outFNTemplate.format(gpmmFileDate, gpmmFileVer)
    outRaster = outDrv.Create(outFN,
                          precipMem.shape[0],precipMem.shape[1],1,gdal.GDT_Float32,
                          ["COMPRESS=LZW","TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES"])
    outRaster.SetGeoTransform(trmmGT)
    outRaster.SetProjection(globalProj)
    outBand = outRaster.GetRasterBand(1)
    # the no data value is prob supposed to be =9999.9 but FP error actually makes it
    outBand.SetNoDataValue(-9999.90039062)
    # the HDFs are in a weird order, columns and rows are transposed and columns are backwards
    outBand.WriteArray(precipMem.T[::-1,::1])
    outBand.FlushCache()
    outBand = None
    outRaster = None